In [13]:
import numpy as np
import scipy.sparse as sps
import h5py
from termcolor import colored
import os

def decode_str(hdf5_str):
    return hdf5_str.tobytes().decode('utf-16')

filename = 'data/Head-and-Neck_10.mat'
f = h5py.File(filename)
name = decode_str(f['patient']['Identifier'][:])
print(colored('Patient: ' + name, 'blue'))

folder = filename.split('.')[0]
if not os.path.exists(folder):
    os.makedirs(folder)

Patient: Head-and-Neck 10


In [14]:
class Region:
    def __init__(self, name):
        self.name = name
        self.D = None # Dose deposition matrix (called A in TROTS)

    def compute_dose(self, x):
        self.dose = self.D.dot(x)
        self.min = self.dose.min()
        self.mean = self.dose.mean()
        self.max = self.dose.max()

def load_rois():
    print('Loading dose deposition matrices...')
    
    rois = {}
    for ref in f['patient']['StructureNames'][:]:
        name = decode_str(f[ref[0]][:])
        rois[name] = Region(name)

    # Get the dose matrix for each ROI
    n_mats = f['data']['matrix']['A'].shape[0]
    for i in range(n_mats):
        name = decode_str(f[f['data']['matrix']['Name'][i,0]][:])

        if name in rois:
            roi = rois[name]
            A = f[f['data']['matrix']['A'][i,0]]
            
            if isinstance(A, h5py.Group): # It's a sparse matrix
                data = np.array(A['data']).ravel()
                ir = np.array(A['ir']).ravel()
                jc = np.array(A['jc']).ravel()
                n_voxels = A.attrs.get('MATLAB_sparse')
                n_beamlets = jc.size - 1
                shape = (n_voxels, n_beamlets)
                
                roi.D = sps.csc_matrix((data, ir, jc), shape=shape)
                
                print(colored(f'  {name} -> {shape} (sparse)', 'green'))
            elif isinstance(A, h5py.Dataset): # It's a dense matrix
                roi.D = A[:].T
                shape = roi.D.shape
                print(colored(f'  {name} -> {shape} (dense)', 'green'))
            else:
                print(colored(f'  {name} -> Not processed!', 'magenta'))
        else:
            print(colored(f'  {name} -> Ignored', 'yellow'))

    # Check that all regions have dose matrices
    for name, roi in rois.items():
        if roi.D is None:
            print(colored(f'  {name} -> No dose matrix!', 'red'))
    
    return rois

rois = load_rois()

Loading dose deposition matrices...
  PTV 0-46 Gy -> (5127, 10579) (sparse)
  Spinal Cord -> (3406, 10579) (sparse)
  Brainstem -> (3674, 10579) (sparse)
  PTV Shell 15 mm -> (4769, 10579) (sparse)
  PTV Shell 30 mm -> (4843, 10579) (sparse)
  PTV Shell 40 mm -> (4700, 10579) (sparse)
  PTV Shell 5 mm -> (4963, 10579) (sparse)
  Patient -> (13512, 10579) (sparse)
  PTV Shell 0 mm -> (4847, 10579) (sparse)
  Smoothing Linear -> Ignored
  Smoothing Quadratic -> Ignored
  Cochlea (R) -> (46, 10579) (sparse)
  Cochlea (L) -> (64, 10579) (sparse)
  Parotid (R) -> (4387, 10579) (sparse)
  Parotid (R) (mean) -> Ignored
  Parotid (L) -> (3630, 10579) (sparse)
  Parotid (L) (mean) -> Ignored
  SMG (R) -> (2294, 10579) (sparse)
  SMG (R) (mean) -> Ignored
  SMG (L) -> (2034, 10579) (sparse)
  SMG (L) (mean) -> Ignored
  Oral Cavity -> (5292, 10579) (sparse)
  Oral Cavity (mean) -> Ignored
  External Ring 20 mm -> (6171, 10579) (sparse)
  Larynx -> (5105, 10579) (sparse)
  Larynx (mean) -> Ignore

In [15]:
import os

def n_voxels(rois):
    n_voxels = 0
    for name, roi in rois.items():
        n_voxels += roi.D.shape[0]
    return n_voxels

def gen_m(folder):
    name = decode_str(f['patient']['Identifier'][:]).replace(' ', '_')
    with open(os.path.join(folder, 'm_' + name + '.txt'), 'w') as file:
        file.write(name + '\n')
        # We don't care about beam geometry for now.
        file.write('1 // Number of beams \n')
        file.write('1 {} \n'.format(int(f['data']['misc']['size'][0][0])))
        file.write('{} // Number of voxels \n'.format(n_voxels(rois)))
        # d_ files will hold float values without need of scaling.
        file.write('1.0 // DoseGridScaling \n'.format(n_voxels(rois)))
        file.write('{} // Number of ROIs \n'.format(len(rois)))
        for i, (name, roi) in enumerate(rois.items()):
            file.write('{} {}\n'.format(2**i, name))
            

def gen_v(folder):
    name = decode_str(f['patient']['Identifier'][:]).replace(' ', '_')
    with open(os.path.join(folder, 'v_' + name + '.txt'), 'w') as file:
        for i, (name, roi) in enumerate(rois.items()):
            print(name)
            voxel_region = 2**i
            for j in range(roi.D.shape[0]):
                file.write('{}\n'.format(voxel_region))
                
gen_m(folder)
gen_v(folder)

Patient
Spinal Cord
Parotid (R)
Parotid (L)
SMG (R)
SMG (L)
MCS
MCM
MCI
MCP
Oesophagus
Brainstem
Oral Cavity
Larynx
Cochlea (R)
Cochlea (L)
PTV 0-46 Gy
PTV Shell 15 mm
PTV Shell 30 mm
PTV Shell 40 mm
PTV Shell 5 mm
PTV Shell 0 mm
External Ring 20 mm


In [16]:
def gen_d(folder):
    name = decode_str(f['patient']['Identifier'][:]).replace(' ', '_')
    
    total_nnz = 0
    for n, roi in rois.items():
        coo = sps.coo_matrix(roi.D)
        total_nnz += coo.nnz
    
    with open(os.path.join(folder, 'd_' + name + '.txt'), 'w') as file:
        file.write('{} \n'.format(total_nnz))
        row_offset = 0
        for i, (n, roi) in enumerate(rois.items()):
            coo = sps.coo_matrix(roi.D)
            print(f'[{i+1:02d}/{len(rois):02d}] {n}: {coo.nnz} nonzeros...')
            for j in range(coo.nnz):
                file.write('{} {} {:.8f}\n'.format(coo.row[j] + row_offset, coo.col[j], coo.data[j]))
            row_offset += coo.shape[0]

gen_d(folder)

[01/23] Patient: 13285073 nonzeros...
[02/23] Spinal Cord: 4475739 nonzeros...
[03/23] Parotid (R): 6169279 nonzeros...
[04/23] Parotid (L): 5486762 nonzeros...
[05/23] SMG (R): 4492607 nonzeros...
[06/23] SMG (L): 3987229 nonzeros...
[07/23] MCS: 2320926 nonzeros...
[08/23] MCM: 2505958 nonzeros...
[09/23] MCI: 3126338 nonzeros...
[10/23] MCP: 3380659 nonzeros...
[11/23] Oesophagus: 1856261 nonzeros...
[12/23] Brainstem: 2342772 nonzeros...
[13/23] Oral Cavity: 8869878 nonzeros...
[14/23] Larynx: 9960419 nonzeros...
[15/23] Cochlea (R): 31540 nonzeros...
[16/23] Cochlea (L): 47205 nonzeros...
[17/23] PTV 0-46 Gy: 9300573 nonzeros...
[18/23] PTV Shell 15 mm: 6221484 nonzeros...
[19/23] PTV Shell 30 mm: 3956424 nonzeros...
[20/23] PTV Shell 40 mm: 2573767 nonzeros...
[21/23] PTV Shell 5 mm: 7807878 nonzeros...
[22/23] PTV Shell 0 mm: 8136885 nonzeros...
[23/23] External Ring 20 mm: 1511145 nonzeros...


In [17]:
def gen_x():
    name = decode_str(f['patient']['Identifier'][:]).replace(' ', '_')
    fluence = f['solutionX'][:].ravel()
    
    with open(os.path.join(folder, 'x_' + name + '.txt'), 'w') as file:
        for x in fluence:
            file.write('{}\n'.format(x))

gen_x()